# Basic data exploration in Python

## Preamble : loading the relevant libraries
* When you progress in programming, you'll see that the phrase "Do not reinvent the wheel" saves you time ! 

* PLUS Python is known as a language with "batteries included"

### Data analysis ![Pandas](/pandas_small.png)
* For data analysis, the reference library is Pandas

* We will explore some functionalities (though this is a library with LOTS of functionalities). More here : [Pandas](https://pandas.pydata.org/docs/index.html)

###  Interactive visualisations ![Plotly](../resources/plotly_graphing_libraries_1.png)
* We will favor Plotly for interactive visualisation though it can be hard to use for beginners, when they start.

* Plotly share some similarities with R's ggplot2, especially the Plotly Express module.

* Plotly can be used with Python, R, Javascript. Plotly is well documented, see here : [Plotly for Python](https://plotly.com/python/)


In [ ]:
import pandas as pd
import plotly.express as px

## Loading data

Data can come from *various sources* :
- tabular data stored on your computer (e.g a .csv or Excel file)
- data stored in Domino Dataset
- file that you pull from Internet (advanced)
- other

Below, we'll see how we can load data stored on file directory and from Domino dataset

### SAS database
YES !!! You can read data from a SAS database.

In [ ]:
# Reading from files
SAS_FILE_PATH = '../data/nhefs.sas7bdat'
#Reading from Domino Dataset
SAS_DATASET = '/domino/datasets/local/NHEFS_SAS/nhefs.sas7bdat'

In [ ]:
df_sas = pd.read_sas(SAS_FILE_PATH, format=None, index=None, encoding=None, chunksize=None, iterator=False)
df_sas.info()

In [ ]:
df_sas_dataset = pd.read_sas(SAS_DATASET, format=None, index=None, encoding=None, chunksize=None, iterator=False)
df_sas_dataset.info()

In [ ]:
df_sas.head()

In [ ]:
df_sas_dataset.head()

### Time series (.csv) stored on your computer

In [ ]:
# Bitcoin data (time series)
data_btc = '/domino/datasets/local/Bitcoin_dataset/Bitbay_BTCEUR_d.csv'
df_btc = pd.read_csv(data_btc, header=1) # the first line is a header

In [ ]:
df_btc.info()

In [ ]:
# df_btc contains time series of Bitcoin/EUR data
# Convert the Date into a datetime object
df_btc["Date"] = pd.to_datetime(df_btc["Date"])

In [ ]:
df_btc.info()

In [ ]:
df_btc.head()

## Selecting, filtering

### Selecting a few columns

In [ ]:
# Pseudo SQL : select seqn, death, sex, age, pregnancies from df_sas
col_selection = ['seqn', 'death', 'sex', 'age', 'pregnancies']
df_sas_light = df_sas[col_selection]
df_sas_light.head()

### Filtering (NHEFS data)

#### 1 or more criteria 

In [ ]:
# Only women
# Pseudo-SQL : select * from df_sas_light where sex = 1
df_sas_women = df_sas_light[df_sas["sex"]==1]

In [ ]:
df_sas_women.head()

In [ ]:
# Only women aged more than 40, from original dataframe
# Pseudo-SQL : select * from df_sas where (sex = 1 and age >= 40)
df_sas_women_above40 = df_sas_light[(df_sas["sex"]==1)&(df_sas["age"]>=40)]
df_sas_women_above40.head()

**FURTHER READING**

You can have a look at Pandas tutorials to go deeper
[Select a subset with Pandas](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html)

## Transforming the data

In [ ]:
# Adding year and month for BTC/EUR data
df_btc["Year"] = df_btc["Date"].dt.year
df_btc["Month"] = df_btc["Date"].dt.month

In [ ]:
# Adding a new column Gender which contains categorical data 
df_sas["gender"] =  df_sas["sex"].apply(lambda x: "MALE" if x ==0 else "FEMALE")
df_sas[["seqn","sex","gender"]].head()

**FURTHER READING**

[Add or create new columns](https://pandas.pydata.org/docs/getting_started/intro_tutorials/05_add_columns.html)

## Summarising and visualising data

### Basic summary statistics

In [ ]:
# First let's select only a few columns
col_selection2 = ['seqn', 'death', 'sex', 'age', 'pregnancies','gender' ]
df_sas_light2 = df_sas[col_selection2]
df_sas_light2.head() 

In [ ]:
# Pandas native function describe()
df_sas_light2.describe(include="all")

Note here that we were right to reencode sex as a categorical data via gender

Further transformation is needed for death and pregnancies which should be viewed as categorical variables.
I leave as an exercise for you to make these transformations 

**EXERCISE**

Further transformation is needed for death and pregnancies which should be viewed as categorical variables.
I leave as an exercise for you to make these transformations

*HINTS* : 
- You can have a look at Pandas documentation here [Categorical data](https://pandas.pydata.org/docs/user_guide/categorical.html)

- For "death" you can do the folowing (see below):

In [ ]:
## Hint for death, you can replicate for "pregnancies" (a bit complex ?)
df_sas_light2["survival"] = df_sas_light2["death"].astype("category") 
df_sas_light2["survival"] = df_sas_light2["survival"].cat.rename_categories([0,1])
df_sas_light2["survival"] = df_sas_light2["survival"].cat.codes

In [ ]:
# Inspect the transformed dataframe
df_sas_light2.head() 

In [ ]:
# Summary statistics "by hand"
# Define the statistics according to the data type
numerical_metrics = ["min", "max", "mean", "median"]

stats_numerical = df_sas_light2.agg({
    "age" : numerical_metrics
})
stats_numerical

In [ ]:
# Summary statistics "by hand"
# Define the statistics according to the data type
categorical_metrics = ["unique"]
stats_categorical = df_sas_light2.agg({
    "gender" : categorical_metrics,
    "survival" : categorical_metrics
})
stats_categorical

In [ ]:
# Summary statistics "by hand"
# Define the statistics according to the data type
df_sas_light2[["gender","survival"]].apply(lambda x: x.value_counts()).T.stack()

### Grouping/aggregating to compute statistics

In [ ]:
# Group by gender
df_grouped = df_sas_light2.groupby(["gender"])
# Average age per gender
df_grouped["age"].mean()
# Average age per gender are not so different

In [ ]:
# Group by gender and survival
# More statistics on age (per gender, survival)
df_grouped = df_sas_light2.groupby(["gender","survival"])
df_grouped["age"].mean()
# Results : as above, survivors or deads whatever their gender have similar average ages

**NOTE** 

- Computing statistics by hand is not easy, neither recommended.

- Use as much as possible the existing Pandas functions (see "Further reading" below)

- We can do more sophisticated aggregations and pivot tables with Pandas

**FURTHER READING**

[Descriptive statistics](https://pandas.pydata.org/docs/user_guide/basics.html#descriptive-statistics)

[Group by (group, split, combine)](https://pandas.pydata.org/docs/user_guide/groupby.html) OR [Comprehensive guide to aggregating, grouping in Pandas](https://pbpython.com/groupby-agg.html)

[Reshaping dataframe, pivot tables](https://pandas.pydata.org/docs/user_guide/reshaping.html)

### Basic data visualisation

##### 1D plots for distributions (histograms, boxplot,..)

In [ ]:
df_sas_light2.info()

In [ ]:
df = df_sas_light2
# Histogram of ages
hist_age = px.histogram(df, x="age")
hist_age.show()

In [ ]:
df = df_sas_light2
# Box plot of ages, depending on gender
box_age_gender = px.box(df, y="age", x="gender", color="gender",
          hover_data=df.columns)
box_age_gender.show()

**NOTE** The box plots confirm that the distribution of age do not differ by gender

#### Plotting time series data

In [ ]:
# Let's now use Plotly
fig = px.line(df_btc, x='Date', y="Open")
fig.show()